### Import packages

In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from pathlib import Path
import pandas as pd
import numpy as np
import random
import glob
import os

from add_features import add_xy_future, add_velocity_xy, add_acceleration_xy, add_average_velocity, add_orientation, add_ball_in_motion, add_distance_to_ball, add_angle_to_ball, add_offside, add_distance_to_onside, add_FM_data, add_tiredness, add_tiredness_short_term
from visualize_game import visualize_game_animation, visualize_prediction_animation
from utils import load_processed_frames, load_FM_data, denominators, google_sheet_to_df, split_match_ids, load_processed_frames, extract_variables, load_tf_model, prepare_EL_input_data, prepare_df, prepare_data, prepare_LSTM_df, prepare_LSTM_input_data, total_error_loss, smooth_predictions_xy, run_model, print_column_variance
from settings import *

2024-05-29 15:10:17.796364: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Necessary columns
| Column Name    | Description                                        |
|----------------|----------------------------------------------------|
| player         | The name of the player                             |
| jersey_number  | The jersey number of the player                    |
| team           | 'home_team', 'away_team', or 'ball'                |
| team_name      | The team of the player                             |
| period         | The period of the game (1 or 2)                    |
| minute         | The minute of the game                             |
| second         | The second within the current minute               |
| frame          | The frame of the game                              |
| distance_ran   | The cumulative distance covered by the player      |
| x              | The x-coordinate of the player                     |
| y              | The y-coordinate of the player                     |

## Define make prediction

In [2]:
# Add all features
def add_all_features(frames_df, match_id):
    # Add the following features
    frames_df = add_xy_future(frames_df, FPS * seconds_into_the_future)
    frames_df = add_velocity_xy(frames_df, 1, smooth=True)
    frames_df = add_acceleration_xy(frames_df, 1, smooth=True)
    frames_df = add_average_velocity(frames_df)
    frames_df = add_orientation(frames_df)
    frames_df = add_ball_in_motion(frames_df)
    frames_df = add_distance_to_ball(frames_df)
    frames_df = add_angle_to_ball(frames_df)
    frames_df = add_distance_to_onside(frames_df)
    frames_df = add_FM_data(frames_df, load_FM_data())
    frames_df = add_tiredness(frames_df)
    frames_df = add_tiredness_short_term(frames_df, window=FPS*20)

    # Add column containing the 'match_id'
    frames_df['match_id'] = match_id

    return frames_df

In [11]:
# Create the vectors 'x_future_pred' and 'y_future_pred' on frames_df
def make_predictions(frames_df, model_name):
    # Prepare the DataFrame by adding all features
    frames_df = add_all_features(frames_df, match_id=1)
    
    # Run the model and add the vectors 'x_future_pred' and 'y_future_pred'
    frames_df = run_model([], model_name, preloaded_frames_df=frames_df)

    # Calculate the error
    error = total_error_loss(frames_df)
    print(f"Error: {error} m")

    # Define the file path
    file_path = f"models/{model_name}.txt"
    # if 'Testing results' does not exists in txt file
    with open(file_path, 'r') as file:
        if 'Testing results' not in file.read():
            # Write the following with f.write
            with open(file_path, 'a') as file:  # 'a' mode to append data
                file.write(f"\nTesting results:\ntest_loss: {error}\n")
            print("Testing results added to the file.")

    return frames_df

### Example usage

In [13]:
# Example match
test_id = 'eaa1aee4-1acf-4a74-acdc-016343b231cc'

# Load the unprocessed version of test_id
file_path = f"{DATA_LOCAL_FOLDER}/data/2023/Allsvenskan/unprocessed/{test_id}.parquet"
unprocessed_frames_df = pd.read_parquet(file_path).iloc[1000:35000]     # Load in 1 minute of unprocessed data

# Make the prediction
model_name = "LSTM_model_v1"
frames_df = make_predictions(unprocessed_frames_df, model_name)

# Create gif file from prediction
visualize_prediction_animation(frames_df, 800, 980, model_name)

194/194 [==============================] - 1s 1ms/step
Error: 1.708 m


## Make Predictions for Thiago's model

In [5]:
# Make predictions for a frame interval
def make_predictions_thiago(match_id, start_frame, end_frame, period, model_name="LSTM_Model"):
    # Load frames
    frames_dfs = load_processed_frames(match_id=match_id)

    # If a match was loaded
    if not frames_dfs:
        raise ValueError(f"Could not load with match_id: {match_id}")

    # Load the frames
    frames_df = frames_dfs[0].copy()

    # Load the correct frame inteval based on period
    if period == 1:
        frames_df = frames_df[(frames_df['frame'] >= start_frame) & (frames_df['frame'] <= end_frame)]
    else:
        # Find max frame period 1 and substract
        max_frame_period_1 = max(frames_df[frames_df['period'] == 1]['frame'])
        frames_df['frame'] = frames_df['frame'] - max_frame_period_1

        frames_df = frames_df[(frames_df['frame'] >= start_frame) & (frames_df['frame'] <= end_frame)]

    # Run the model and add the vectors 'x_future_pred' and 'y_future_pred'
    frames_df = run_model([], model_name, preloaded_frames_df=frames_df)

    return frames_df

# Make predictions for a signle frame
def make_predictions_thiago_frame(match_id, frame, period, model_name="LSTM_Model"):
    # Load frames
    frames_dfs = load_processed_frames(match_id=match_id)

    # If a match was loaded
    if not frames_dfs:
        raise ValueError(f"Could not load with match_id: {match_id}")

    # Load the frames
    frames_df = frames_dfs[0].copy()

    # Load the correct frame inteval based on period
    if period == 2:
        # Find max frame period 1 and substract
        max_frame_period_1 = max(frames_df[frames_df['period'] == 1]['frame'])
        frames_df['frame'] = frames_df['frame'] - max_frame_period_1

    # Load a whole sequence if we can a LSTM model
    if 'LSTM' in model_name:
        frames_df = frames_df[(frames_df['frame'] > frame - 50) & (frames_df['frame'] <= frame)]
    else:
        frames_df = frames_df[frames_df['frame'] == frame]

    # Run the model and add the vectors 'x_future_pred' and 'y_future_pred'
    frames_df = run_model([], model_name, preloaded_frames_df=frames_df)

    # Only keep the last frame
    max_frame = max(frames_df['frame'])
    frames_df = frames_df[frames_df['frame'] == max_frame]

    return frames_df

In [7]:
# Example match_id from Singality
test_id = 'eaa1aee4-1acf-4a74-acdc-016343b231cc'

# Run prediction interval of frames
start_frame = 500
end_frame = 1500
# frames_df = make_predictions_thiago(test_id, start_frame, end_frame, 1, model_name="LSTM_Model")

# Run prediction on single frame
frame = 500
frames_df = make_predictions_thiago_frame(test_id, frame, 1, model_name="LSTM_Model")

frames_df

1/1 [==============================] - 0s 355ms/step


,x,y,v_x,v_y,distance_to_ball,v_x_avg,v_y_avg,a_x,a_y,tiredness,...,team,team_name,frame,ball_in_motion,minute,second,x_future,y_future,x_future_pred,y_future_pred
9,83.639999,65.059998,-0.23,-0.75,0.0,-0.53,0.36,-0.03,-0.01,0.016424,...,home_team,AIK,500,False,0,20,83.05,64.84,83.549736,64.191010
19,80.190002,52.320000,-1.00,-0.25,0.0,-0.53,0.36,0.01,-0.02,0.012764,...,home_team,AIK,500,False,0,20,78.10,52.57,78.549194,51.886681
29,80.590004,32.930000,-0.02,1.27,0.0,-0.53,0.36,-0.00,-0.00,0.010864,...,home_team,AIK,500,False,0,20,79.05,34.70,80.249870,35.057995
39,63.899998,57.950001,-1.73,-0.25,0.0,-0.53,0.36,-0.02,0.05,0.014186,...,home_team,AIK,500,False,0,20,64.74,58.64,61.995407,58.184921
49,71.459999,59.870003,-1.02,0.77,0.0,-0.53,0.36,0.00,-0.04,0.015837,...,home_team,AIK,500,False,0,20,69.65,62.08,69.704430,60.237938
59,51.309998,51.549999,-0.27,0.98,0.0,-0.53,0.36,0.01,0.04,0.017976,...,home_team,AIK,500,False,0,20,50.74,53.46,50.934105,53.113056
69,101.050003,43.470001,-0.25,-1.48,0.0,-0.53,0.36,0.00,-0.07,0.012845,...,home_team,AIK,500,False,0,20,99.16,42.24,100.226799,41.168949
79,55.840000,65.980003,-1.75,0.75,0.0,-0.53,0.36,-0.06,0.03,0.018477,...,home_team,AIK,500,False,0,20,52.34,66.26,53.742138,66.404716
89,46.590000,58.509998,-0.75,1.52,0.0,-0.53,0.36,-0.03,-0.04,0.022200,...,home_team,AIK,500,False,0,20,45.89,60.42,45.195362,61.511837
99,53.090000,30.490000,-0.48,-0.50,0.0,-0.53,0.36,-0.02,-0.02,0.015190,...,home_team,AIK,500,False,0,20,53.08,30.81,52.742744,29.910585
